In [83]:
!pip -q install accelerate peft bitsandbytes trl py7zr auto-gptq optimum transformers

In [84]:
pip show accelerate peft bitsandbytes trl py7zr auto-gptq optimum transformer

In [81]:
!pip list

In [2]:
from huggingface_hub import notebook_login

In [4]:
notebook_login()

In [5]:
import torch

In [6]:
from datasets import load_dataset,Dataset

In [7]:
df=load_dataset("samsum",split="train")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


The repository for samsum contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/samsum.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Generating train split:   0%|          | 0/14732 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/819 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/818 [00:00<?, ? examples/s]

In [20]:
df.shape

(14732, 3)

In [21]:
df

Dataset({
    features: ['id', 'dialogue', 'summary'],
    num_rows: 14732
})

In [22]:
datadf=df.to_pandas()

In [23]:
datadf.head()

,id,dialogue,summary
0,13818513,Amanda: I baked cookies. Do you want some?\r\...,Amanda baked cookies and will bring Jerry some...
1,13728867,Olivia: Who are you voting for in this electio...,Olivia and Olivier are voting for liberals in ...
2,13681000,"Tim: Hi, what's up?\r\nKim: Bad mood tbh, I wa...",Kim may try the pomodoro technique recommended...
3,13730747,"Edward: Rachel, I think I'm in ove with Bella....",Edward thinks he is in love with Bella. Rachel...
4,13728094,Sam: hey overheard rick say something\r\nSam:...,"Sam is confused, because he overheard Rick com..."


In [24]:
datadf=datadf.sample(20)

In [25]:
datadf["text"]=datadf[["dialogue", "summary"]].apply(lambda x: "###Human: Summarize this following dialogue: " + x["dialogue"] + "\n###Assistant: " +x["summary"], axis=1 )

In [26]:
datadf.head()

,id,dialogue,summary,text
10491,13716668,Bret: two first people to reply are getting a ...,Kit and Gabe were first to reply to Bret so th...,###Human: Summarize this following dialogue: B...
10938,13865094,Amy: If anyone has a spare ticket for grad hal...,Amy and Lilly are looking for tickets for gran...,###Human: Summarize this following dialogue: A...
6256,13730071,Sarah: In the shops now.\r\nSarah: Is there an...,Barb would like Sarah to buy some pumpkin or c...,###Human: Summarize this following dialogue: S...
4777,13680844,Barbara: Heyyy\r\nJohn: Hullo\r\nBarbara: Are ...,Barbara invited John for drinks at Scolt's Hea...,###Human: Summarize this following dialogue: B...
10781,13681100,Serena: Julie you looked so lovely last night ...,Serena and Julie had a great time last night. ...,###Human: Summarize this following dialogue: S...


In [27]:
datadf["text"].iloc[0]

"###Human: Summarize this following dialogue: Bret: two first people to reply are getting a free copy of a game on steam lol my brother botched it when buying a coop game for us\r\nKit: ngdfngdgndfg\r\nGabe: here I am!\r\nGabe: gamer Gabe, the Gaber!\r\nBret: hhahah congrats guyz i'll send you the codes\r\nDan: why am i always late for every-fukken-thing?!!?!?\n###Assistant: Kit and Gabe were first to reply to Bret so they are getting a free copy of a game on steam from him. Dan is dissapointed he was late."

In [28]:
datadf.head(1)

,id,dialogue,summary,text
10491,13716668,Bret: two first people to reply are getting a ...,Kit and Gabe were first to reply to Bret so th...,###Human: Summarize this following dialogue: B...


In [29]:
data=Dataset.from_pandas(datadf)

In [30]:
data

Dataset({
    features: ['id', 'dialogue', 'summary', 'text', '__index_level_0__'],
    num_rows: 20
})

In [31]:
from transformers import AutoModelForCausalLM, AutoTokenizer, GPTQConfig, TrainingArguments

In [32]:
tokenizer=AutoTokenizer.from_pretrained("TheBloke/Mistral-7B-Instruct-v0.1-GPTQ")

tokenizer_config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

In [33]:
tokenizer.eos_token

'</s>'

In [34]:
tokenizer.pad_token=tokenizer.eos_token

In [35]:
quantization_config_loading = GPTQConfig(bits=4, disable_exllama=True, tokenizer=tokenizer)

Using `disable_exllama` is deprecated and will be removed in version 4.37. Use `use_exllama` instead and specify the version with `exllama_config`.The value of `use_exllama` will be overwritten by `disable_exllama` passed in `GPTQConfig` or stored in your config file.


In [36]:
model=AutoModelForCausalLM.from_pretrained( "TheBloke/Mistral-7B-Instruct-v0.1-GPTQ",
                                     quantization_config=quantization_config_loading,
                                      device_map="auto")

config.json:   0%|          | 0.00/963 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/quantizers/auto.py:167: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` from the model will be used.However, loading attributes (e.g. ['use_cuda_fp16', 'use_exllama', 'max_input_length', 'exllama_config', 'disable_exllama']) will be overwritten with the one you passed to `from_pretrained`. The rest will be ignored.
  warnings.warn(warning_msg)


model.safetensors:   0%|          | 0.00/4.16G [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:4565: FutureWarning: `_is_quantized_training_enabled` is going to be deprecated in transformers 4.39.0. Please use `model.hf_quantizer.is_trainable` instead
  warnings.warn(
Some weights of the model checkpoint at TheBloke/Mistral-7B-Instruct-v0.1-GPTQ were not used when initializing MistralForCausalLM: ['model.layers.0.mlp.down_proj.bias', 'model.layers.0.mlp.gate_proj.bias', 'model.layers.0.mlp.up_proj.bias', 'model.layers.0.self_attn.k_proj.bias', 'model.layers.0.self_attn.o_proj.bias', 'model.layers.0.self_attn.q_proj.bias', 'model.layers.0.self_attn.v_proj.bias', 'model.layers.1.mlp.down_proj.bias', 'model.layers.1.mlp.gate_proj.bias', 'model.layers.1.mlp.up_proj.bias', 'model.layers.1.self_attn.k_proj.bias', 'model.layers.1.self_attn.o_proj.bias', 'model.layers.1.self_attn.q_proj.bias', 'model.layers.1.self_attn.v_proj.bias', 'model.layers.10.mlp.down_proj.bias', 'model.layers.10.mlp.gate_proj.bias', 'model.la

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [37]:
print(model)

In [38]:
model.config.use_cache=False
model.config.pretraining_tp=1
model.gradient_checkpointing_enable()

In [39]:
from peft import prepare_model_for_kbit_training

In [40]:
model = prepare_model_for_kbit_training(model)

In [41]:
from peft import LoraConfig

In [42]:
peft_config=LoraConfig(
    r=16,lora_alpha=16,lora_dropout=0.05, bias="none", task_type="CAUSAL_LM", target_modules= ["q_proj", "v_proj"]
)

In [43]:
from peft import get_peft_model

In [44]:
model=get_peft_model(model,peft_config)

In [45]:
from trl import SFTTrainer

In [46]:
from transformers import TrainingArguments

In [49]:
training_arguments = TrainingArguments(
        output_dir="mistral-finetuned-samsum1",
        per_device_train_batch_size=8,
        gradient_accumulation_steps=1,
        optim="paged_adamw_32bit",
        learning_rate=2e-4,
        lr_scheduler_type="cosine",
        save_strategy="epoch",
        logging_steps=5,
        num_train_epochs=1,
        max_steps=25,
        fp16=True,
        #push_to_hub=True
    )

In [50]:
notebook_login()

In [ ]:
# model

In [51]:
trainer=SFTTrainer(
    model=model,
    train_dataset=data,
    peft_config=peft_config,
    dataset_text_field="text",
    args=training_arguments,
    tokenizer=tokenizer,
    packing=False,
    max_seq_length=512
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field, max_seq_length. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:280: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:318: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(


Map:   0%|          | 0/20 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:408: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(
max_steps is given, it will override any value given in num_train_epochs


In [52]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
5,2.263800
10,1.899800
15,1.757300
20,1.605400
25,1.640700


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning

TrainOutput(global_step=25, training_loss=1.8334238815307617, metrics={'train_runtime': 243.6195, 'train_samples_per_second': 0.821, 'train_steps_per_second': 0.103, 'total_flos': 58727600750592.0, 'train_loss': 1.8334238815307617, 'epoch': 8.333333333333334})

In [53]:
trainer.push_to_hub()

events.out.tfevents.1721889607.d7db8d17e65a.941.0:   0%|          | 0.00/7.18k [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/27.3M [00:00<?, ?B/s]

Upload 4 LFS files:   0%|          | 0/4 [00:00<?, ?it/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/5.37k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Venky2409/mistral-finetuned-samsum1/commit/84e76b15c636adb23e7808908a0a40a97a2c314f', commit_message='End of training', commit_description='', oid='84e76b15c636adb23e7808908a0a40a97a2c314f', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
! cp -r /content/mistral-finetuned-samsum /content/drive/MyDrive/

# Inferencing

In [54]:
from peft import AutoPeftModelForCausalLM
from transformers import GenerationConfig
from transformers import AutoTokenizer
import torch

In [55]:
tokenizer = AutoTokenizer.from_pretrained("/content/mistral-finetuned-samsum1")

In [56]:
inputs = tokenizer("""
###Human: Summarize this following dialogue: Vasanth: I'm at the railway station in Chennai Karthik: No problems so far? Vasanth: no, everything's going smoothly Karthik: good. lets meet there soon!
###Assistant: """, return_tensors="pt").to("cuda")


In [57]:
model = AutoPeftModelForCausalLM.from_pretrained(
    "/content/mistral-finetuned-samsum1",
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map="cuda")

/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:4565: FutureWarning: `_is_quantized_training_enabled` is going to be deprecated in transformers 4.39.0. Please use `model.hf_quantizer.is_trainable` instead
  warnings.warn(
Some weights of the model checkpoint at TheBloke/Mistral-7B-Instruct-v0.1-GPTQ were not used when initializing MistralForCausalLM: ['model.layers.0.mlp.down_proj.bias', 'model.layers.0.mlp.gate_proj.bias', 'model.layers.0.mlp.up_proj.bias', 'model.layers.0.self_attn.k_proj.bias', 'model.layers.0.self_attn.o_proj.bias', 'model.layers.0.self_attn.q_proj.bias', 'model.layers.0.self_attn.v_proj.bias', 'model.layers.1.mlp.down_proj.bias', 'model.layers.1.mlp.gate_proj.bias', 'model.layers.1.mlp.up_proj.bias', 'model.layers.1.self_attn.k_proj.bias', 'model.layers.1.self_attn.o_proj.bias', 'model.layers.1.self_attn.q_proj.bias', 'model.layers.1.self_attn.v_proj.bias', 'model.layers.10.mlp.down_proj.bias', 'model.layers.10.mlp.gate_proj.bias', 'model.la

In [58]:
generation_config = GenerationConfig(
    do_sample=True,
    top_k=1,
    temperature=0.1,
    max_new_tokens=25,
    pad_token_id=tokenizer.eos_token_id
)

In [68]:
generation_config

GenerationConfig {
  "do_sample": true,
  "max_new_tokens": 25,
  "pad_token_id": 2,
  "temperature": 0.1,
  "top_k": 1
}

In [75]:
import time
st_time = time.time()
outputs = model.generate(**inputs, generation_config=generation_config)
result = tokenizer.decode(outputs[0], skip_special_tokens=True)
time.time()-st_time

28.90431547164917

In [76]:
result

"\n###Human: Summarize this following dialogue: Vasanth: I'm at the railway station in Chennai Karthik: No problems so far? Vasanth: no, everything's going smoothly Karthik: good. lets meet there soon!\n###Assistant:  Vasanth is at the railway station in Chennai and everything is going smoothly. He will meet Karthik there soon"

In [61]:
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

In [72]:
tokenizer.decode(outputs[0], skip_special_tokens=True)

"\n###Human: Summarize this following dialogue: Vasanth: I'm at the railway station in Chennai Karthik: No problems so far? Vasanth: no, everything's going smoothly Karthik: good. lets meet there soon!\n###Assistant:  Vasanth is at the railway station in Chennai and everything is going smoothly. He will meet Karthik there soon"

In [74]:
print(time.time()-st_time)